In [86]:
import yfinance as yf

import pandas as pd

import plotly.graph_objects as go

import plotly.express as px

from sklearn.preprocessing import MinMaxScaler

import keras

import itertools

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.layers import Dense
from keras.utils import to_categorical
from keras.optimizers import SGD,Adadelta,Adam,RMSprop 
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
from keras.layers import Dropout


url="https://raw.githubusercontent.com/albbano-lee/financial-data/refs/heads/main/tickers.csv"

data=pd.read_csv(url)

data_head=data.head() 

tickers=data["Symbol"].tolist()

missing_data=['KSPI', 'BTSGU', 'ALAB', 'LINE']

for missed in missing_data:
    tickers.remove(missed)

print(data_head)

print(tickers)

  Symbol                                  Name Last Sale  Net Change % Change  \
0   AAON                AAON Inc. Common Stock  $132.315      -0.865  -0.649%   
1   AAPL               Apple Inc. Common Stock   $224.99      -3.230  -1.415%   
2   ABNB      Airbnb Inc. Class A Common Stock   $132.69      -1.870   -1.39%   
3   ACGL  Arch Capital Group Ltd. Common Stock   $101.43       1.380   1.379%   
4   ADBE               Adobe Inc. Common Stock   $499.88     -29.990   -5.66%   

     Market Cap        Country  IPO Year    Volume       Sector  \
0  1.075451e+10  United States       NaN     66735  Industrials   
1  3.400909e+12  United States    1980.0  13790936   Technology   
2  8.412170e+10  United States    2020.0    697404      Finance   
3  3.816221e+10        Bermuda       NaN    523129      Finance   
4  2.200472e+11  United States    1986.0   1880614   Technology   

                                  Industry  
0          Industrial Machinery/Components  
1                   

In [20]:
stock_data=yf.download(tickers, start="2013-01-01", end="2023-12-31", group_by="ticker")

closing_prices=pd.DataFrame()

selected_tickers=tickers[ : 5]

for ticker in selected_tickers:
    closing_prices[ticker]=stock_data[ticker]["Close"]
    
fig = px.line(
    closing_prices,
    title="NASDAQ selected Companies Closing Prices (2013-2023)",
    labels={"value": "Price (＄)", "index": "year"},
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Price (＄)",
    legend_title="Company Ticker",
    template="plotly_dark"
)

fig.show()

[*********************100%***********************]  274 of 274 completed


In [109]:
start_date="2013-01-01"

end_date="2023-12-31"

stock_data_dict={}

open_price=pd.DataFrame()

close_price=pd.DataFrame()

for ticker in tickers:
    
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    
    stock_data_dict[ticker] = stock_data
    
    open_price[ticker]=stock_data_dict[ticker]["Open"]
    
    close_price[ticker]=stock_data_dict[ticker]["Close"]

stock_data_dict['AAPL']

open_price.fillna(method="bfill", inplace=True)

close_price.fillna(method="bfill", inplace=True)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Using Sde sampling to train

In [113]:
open_price

,AAON,AAPL,ABNB,ACGL,ADBE,ADI,ADP,ADSK,AEP,AFRM,...,WMG,WTW,WWD,WYNN,XEL,Z,ZBRA,ZG,ZM,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02 00:00:00+00:00,6.337778,19.779285,146.000000,14.793333,37.919998,42.799999,50.974541,36.049999,43.169998,90.900002,...,27.000000,89.774834,39.169998,116.370003,27.030001,27.650000,40.040001,8.815092,65.000000,27.500000
2013-01-03 00:00:00+00:00,6.296296,19.567142,146.000000,14.756667,38.139999,43.599998,51.457420,36.750000,43.650002,90.900002,...,27.000000,90.940399,39.730000,118.570000,27.389999,27.650000,40.790001,8.883692,65.000000,27.500000
2013-01-04 00:00:00+00:00,6.441481,19.177500,146.000000,14.770000,37.880001,43.230000,51.641792,36.529999,43.639999,90.900002,...,27.000000,91.920532,39.310001,119.930000,27.330000,27.650000,40.509998,8.855628,65.000000,27.500000
2013-01-07 00:00:00+00:00,6.402963,18.642857,146.000000,14.780000,37.810001,42.009998,51.641792,36.000000,43.430000,90.900002,...,27.000000,91.311256,38.980000,120.730003,27.280001,27.650000,40.570000,9.276582,65.000000,27.500000
2013-01-08 00:00:00+00:00,6.385185,18.900356,146.000000,14.726667,37.779999,42.610001,51.615452,36.150002,43.270000,90.900002,...,27.000000,91.258278,38.750000,121.040001,27.070000,27.650000,40.650002,9.834737,65.000000,27.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00+00:00,72.639999,195.179993,141.880005,74.220001,600.799988,197.619995,228.990005,243.740005,80.570000,49.000000,...,35.299999,238.710007,134.539993,90.000000,61.980000,59.040001,266.769989,57.900002,72.120003,221.669998
2023-12-26 00:00:00+00:00,74.000000,193.610001,141.000000,74.000000,598.919983,197.899994,231.589996,242.490005,80.500000,49.349998,...,35.230000,240.350006,134.899994,90.570000,61.980000,59.099998,270.000000,58.290001,72.610001,222.225006
2023-12-27 00:00:00+00:00,74.440002,192.490005,139.134995,73.250000,598.599976,198.869995,231.889999,245.360001,80.709999,50.200001,...,35.180000,238.529999,136.289993,91.000000,61.730000,59.000000,276.500000,57.689999,74.300003,225.160004


In [111]:
close_price

,AAON,AAPL,ABNB,ACGL,ADBE,ADI,ADP,ADSK,AEP,AFRM,...,WMG,WTW,WWD,WYNN,XEL,Z,ZBRA,ZG,ZM,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02 00:00:00+00:00,6.302222,19.608213,144.710007,14.793333,38.340000,44.000000,51.264267,36.939999,43.650002,96.364998,...,30.120001,90.754967,39.759998,118.070000,27.379999,27.740000,40.959999,8.824447,62.000000,33.000000
2013-01-03 00:00:00+00:00,6.411852,19.360714,144.710007,14.750000,37.750000,43.290001,51.466198,36.369999,43.619999,96.364998,...,30.120001,91.682121,39.009998,119.339996,27.299999,27.740000,41.000000,8.924228,62.000000,33.000000
2013-01-04 00:00:00+00:00,6.444444,18.821428,144.710007,14.876667,38.130001,42.520000,51.922737,36.360001,43.549999,96.364998,...,30.120001,91.655632,39.330002,120.779999,27.400000,27.740000,40.669998,9.229810,62.000000,33.000000
2013-01-07 00:00:00+00:00,6.373333,18.710714,144.710007,14.730000,37.939999,42.650002,51.720806,36.090000,43.369999,96.364998,...,30.120001,91.390732,38.810001,121.190002,27.110001,27.740000,40.900002,9.853446,62.000000,33.000000
2013-01-08 00:00:00+00:00,6.388148,18.761070,144.710007,14.750000,38.139999,42.209999,52.019314,36.299999,43.150002,96.364998,...,30.120001,90.940399,38.759998,121.089996,27.160000,27.740000,40.930000,10.015591,62.000000,33.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00+00:00,73.410004,193.600006,140.800003,73.839996,598.750000,197.100006,230.970001,242.759995,80.379997,49.060001,...,35.290001,240.089996,135.110001,90.419998,62.060001,58.790001,269.410004,58.070000,72.500000,221.679993
2023-12-26 00:00:00+00:00,74.150002,193.050003,138.720001,73.510002,598.260010,198.869995,232.539993,245.070007,80.730003,50.040001,...,35.139999,239.089996,135.399994,91.000000,62.250000,58.680000,275.500000,57.759998,74.209999,224.589996
2023-12-27 00:00:00+00:00,74.209999,193.149994,136.550003,73.540001,596.080017,199.350006,231.610001,245.110001,80.809998,51.240002,...,35.459999,239.300003,136.029999,90.629997,61.540001,59.470001,275.790009,58.509998,73.730003,225.020004


AAON    0
AAPL    0
ABNB    0
ACGL    0
ADBE    0
       ..
Z       0
ZBRA    0
ZG      0
ZM      0
ZS      0
Length: 274, dtype: int64
